## Example

I've waffled on what example to use for ``wig``/``MCNP`` for a while.  All the nuclear stuff that isn't boring is, in general, classified or difficult.  So, I went for the "hello world" of modeling - modeling the text ``wig``.

The first thing we have to do to start the simulation is to import ``wig``.  This is reasonably simple: ``wig`` has a main class, called ``wig.wig`` and then seven other classes, imported below, named (hopefully) self-explanatorially. We should also import ``numpy``, because I can't imagine life without it.

In [1]:
from wig import wig, geo, cell, matl, phys, source, tally
import numpy as np

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
scene = wig.wig(filename='wig_logo', flavor='6',
                comment='''The text wig written in hacky/blocky font''',
                render=True)

In [3]:
wu1 = geo.geo().rpp(x=[-70., -60.], y=[-5.0, 5.0], z=[0., 70.], id='w_upright_1')
wu2 = geo.geo().rpp(x=[-50., -40.], y=[-5.0, 5.0], z=[0., 40.], id='w_upright_2')
wu3 = geo.geo().rpp(x=[-30., -20.], y=[-5.0, 5.0], z=[0., 70.], id='w_upright_3')
wbase = geo.geo().rpp(x=[-70., -20.], y=[-5.0, 5.0], z=[0., 10.], id='w_base')
ibase = geo.geo().rcc(c=(0., 0., 0.), lz=50.0, r=5.0, id='i_base')
idot = geo.geo().sph(c=(0., 0., 60.), r=5.0, id='i_dot')
gout = geo.geo().rcc(c=(40., -5.0, 25.), r=25.0, ly=10.0, id='gout')
gin = geo.geo().rcc(c=(40., -5.0 - 1., 25.), r=15.0 , ly=10.0 + 2., id='gin')
gtail1 = geo.geo().rpp(x=[55., 65.], y=[-5., 5.], z=[-25., 0.0], id='gtail1')
gtail2 = geo.geo().rpp(x=[55., 15.], y=[-5., 5.], z=[-25., -15.], id='gtail2')
floor = geo.geo().rpp(x=[-100., 15.], y=[-50., 50.], z=[-50., 0.], id='floor')
universe = geo.geo().sph(c=(0., 0., 0.), r=250., id='universe')
scene.geo([wu1, wu2, wu3, wbase, ibase, idot, gout, gin, floor, gtail1, gtail2, universe])

In [4]:
al = matl.matl(id='aluminum', rho=40.0, atom_perc=[('Al-27', 1.0)], color='#3b3c5a')
concrete = matl.matl(id='concrete', rho=40.0, atom_perc=[('U-235', 1.0)], color='#a7a9ac')
tungsten = matl.matl(id='tungsten', rho=19.3, color='#746660',
                     atom_perc=[('W-182', 0.2650), ('W-183', 0.1431),
                                ('W-184', 0.3064), ('W-186', 0.2843)])
void = matl.air()
scene.matl([al, concrete, tungsten, void])

In [5]:
wcell = cell.cell(wbase + wu1 + wu2 + wu3, al)
gcell = cell.cell((gout + gtail1 + gtail2) - gin, tungsten)
scene.cell([wcell, gcell,
            ibase.cell(al), idot.cell(al), floor.cell(concrete)], universe_matl=void)

In [6]:
physics = phys.phys(particles='n', ctme=30., minE=1.0E-3, maxE=20.0)
scene.phys(physics)

In [7]:
planesrc = source.source(shape='plane', lx=[-60., 65.], ly=[-45.], lz=[0., 70.],
                         direction='+y', particle='n', spectrum=15.0, id='planesrc')
scene.source([planesrc])

In [8]:
back_plane = tally.tally(particle='n', comment='plane')\
    .mesh_tally(xmin=-80.0, xmax=80.0, i=160, ymin=95., ymax=105., j=1,
                zmin=-30.0, zmax=80.0, k=110)
scene.tally([back_plane])

In [9]:
scene.run(clean=True, render=True, camera_location=(200., -200., 200.))

ampersand
sdef    par=n vec=0 1 0 dir=1 X=d1 Y=-45.000000 Z=d2 erg=1.50 |	sdef    par=n vec=0 -1 0 dir=1 X=d1 Y=-45.000000 Z=d2 erg=7.0
cut:n j 1.000000e-03					      |	cut:n j 1.000000e-08                                         
m2    92235 1.0000000000e+00				      |	m2     6012 1.0000000000e+00                                 

4
The differences in between files is :
sdef    par=n vec=0 1 0 dir=1 X=d1 Y=-45.000000 Z=d2 erg=1.50 |	sdef    par=n vec=0 -1 0 dir=1 X=d1 Y=-45.000000 Z=d2 erg=7.0
cut:n j 1.000000e-03					      |	cut:n j 1.000000e-08                                         
m2    92235 1.0000000000e+00				      |	m2     6012 1.0000000000e+00                                 

So running the MCNP deck


['nohup', 'mcnp6', 'inp=wig_logo.inp', 'out=wig_logo.out', 'run=wig_logo_runtpe', 'mctal=wig_logo_tallies.out', 'meshtal=wig_logo.meshtal', 'tasks 3']
